### Pre-train BERT
* c09k_BERT_pretrain 코드에 이어, max_length를 512로 늘려서 pretrain 추가 수행
* https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python
* https://huggingface.co/transformers/v3.2.0/training.html

In [1]:
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
import nltk
from nltk.data import load
import pickle

/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Tokenizer train data 생성
* 약어 이후에 등장하는 마침표를 사용해 문장이 분리되지 않도록 조치를 해야 한다.
* NLTK의 tokenizer를 사용해 문장 분리하기 위해 extra_abbreviations에 예외조건을 추가하여 준다.
* 클린징이 끝난 데이터는 토크나이저 학습에 바로 사용하고, 이 데이터에서 테스트셋을 분리하여 pre-train 데이터로 사용예정
    * https://cryptosalamander.tistory.com/140?category=1218889

In [2]:
sent_tokenizer = load("tokenizers/punkt/english.pickle")
extra_abbreviations = [
    'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
    'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
    'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
    '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
    'P','Q','R','S','T','U','V','W','X','Y','Z']
sent_tokenizer._params.abbrev_types.update(extra_abbreviations)

In [3]:
# NLTK의 tokenizer를 사용해 문장 분리(미사용)
# https://cryptosalamander.tistory.com/140?category=1218889
def sent_tokenize(input='./input.txt', output='./output.txt'):
    sent_tokenizer = load("tokenizers/punkt/english.pickle")
    extra_abbreviations = [
        'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
        'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
        'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
        '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
        'P','Q','R','S','T','U','V','W','X','Y','Z']
    sent_tokenizer._params.abbrev_types.update(extra_abbreviations)

    load_file=open(input,'r')
    save_file=open(output,'w')
    no_blank = False
    while True:
        line = load_file.readline()
        if line == "":
            break
        if line.strip() == "":
            if no_blank:
                continue
            save_file.write(f"{line}")
        else:
            print(line)
            result_ = tokenizer.tokenize(line)
            print(result_)
            result  = [ f"{cur_line}\n" for cur_line in result_ ]
            for save_line in result:
                save_file.write(save_line)

In [4]:
try:
    dataset = Dataset.from_text('data/c09k_corpus.txt')
except:
    kiwee_files = ['data/c09k_0001-1000.xlsx', 'data/c09k_1001-2000.xlsx', 'data/c09k_2001-3000.xlsx', 'data/c09k_3001-3935.xlsx']
    with open('data/c09k_corpus.txt', 'a') as f:
        f.truncate(0)
        for i, fn in enumerate(kiwee_files):
            tmp = pd.read_excel(fn).fillna('')
            # pandas는 비어있는 컬럼의 dtype을 float로 바꿔서 인식한다. 그로 인해 토크나이징 할 데이터가 없으면 오류가 발생되어 fillna를 사용해 모두 텍스트로 인식시키도록 한다
            # https://stackoverflow.com/questions/53953286/pandas-read-excel-blanks-in-string-columns-convert-to-floats-converting-via-st
            col_text = ['발행번호', '발명의명칭', '요약', '대표청구항', '과제', '해결방안']
            tmp = tmp[col_text]
            for index, row in tmp.iterrows():
        #         print(index, '\n', row['발명의명칭'], row['요약'], row['대표청구항'], row['과제'], row['해결방안'], '\n')
                for col in col_text[1:]:
        #             print('처리중인 데이터:', col, row[col], '\n')
                    if row[col].strip() == "":
                        pass
                    else:
        #                 print(row[col].strip())
                        row[col] = unicodedata.normalize('NFKC', row[col])
                        # row[col] = unicodedata.normalize('NFC', row[col])  # 자음과 모음이 깨질 때는 NFC로 변환
                        # NFD(Normalization Form Decomposition) : 자음과 모음이 분리
                        # row[col] = unicodedata.normalize('NFKD', row[col])
                        #     https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
                        row[col] = row[col].replace('\n\t',' ')
                        row[col] = row[col].replace('\n',' ')
                        row[col] = row[col].replace('&lt;',' ')
                        row[col] = row[col].replace('_x000d_',' ')
                        row[col] = row[col].replace('\t\t',' ')
                        row[col] = row[col].replace('@@',' ')
                        row[col] = row[col].replace('.  .','.')
                        row[col] = row[col].replace('. .','.')
                        row[col] = row[col].replace('..','.')
                        row[col] = row[col].replace('〜','~')
                        row[col] = row[col].replace(' . ','.')
                        row[col] = row[col].replace(' ． ','.')
                        row[col] = row[col].replace('． ','.')
                        row[col] = row[col].replace('. ','.')
                        row[col] = row[col].replace('  ',' ')
                        row[col] = row[col].replace('  ',' ')
                        row[col] = row[col].replace('【과제】',' ')
                        row[col] = row[col].replace('【요약】',' ')
                        row[col] = row[col].replace('【해결 수단】',' ')
                        str_tmp = sent_tokenizer.tokenize(row[col].strip())
        #                 print('문장 분리: ', str_tmp, '\n'*3)
        #                 result  = [f"{line}\n" for line in str_tmp]
                        for line in str_tmp:
                            f.write(f"{line}\n")
    dataset = Dataset.from_text('data/c09k_corpus.txt')

### 학습 데이터 분리/로딩

In [5]:
try:
    with open('data/c09k_dataset.pkl','rb') as f:
        d = pickle.load(f)
    print('dataset loading completed')
except:
    d = dataset.train_test_split(test_size=0.1)
    with open('data/c09k_dataset.pkl','wb') as f:
        pickle.dump(d, f)
    print('dataset split/saving completed')    

dataset loading completed


In [6]:
# # d["train"], d["test"]
# for t in d["train"]["text"][:3]:
#     print(t)
#     print("="*50)

In [7]:
# special_tokens = [
#   "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
# ]
# # if you want to train the tokenizer on both sets
# # files = ["train.txt", "test.txt"]
# # training the tokenizer on the training set
# files = ["data/c09k_corpus.txt"]
# # 30,522 vocab is BERT's default vocab size, feel free to tweak
# vocab_size = 8000
# # maximum sequence length, lowering will result to faster training (when increasing batch size)
# max_length = 512
# # whether to truncate
# truncate_longer_samples = False

In [8]:
model_path = "c09k_pretrained_bert"
vocab_size = 8000
max_length = 512
truncate_longer_samples = False

### Pre-train data 생성

In [9]:
# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
    tokenizer_cfg = {"do_lower_case": True,
                     "unk_token": "[UNK]",
                     "sep_token": "[SEP]",
                     "pad_token": "[PAD]",
                     "cls_token": "[CLS]",
                     "mask_token": "[MASK]",
                     "model_max_length": max_length,
                     "max_len": max_length,
                    }
    json.dump(tokenizer_cfg, f)

### Tokenizing the Dataset

In [10]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
# tokenizer = BertTokenizer.from_pretrained(model_path, )

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file c09k_pretrained_bert/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 512,
  "max_position_embeddings": 512,
  "model_max_length": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sep_token": "[SEP]",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 30522
}



vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [11]:
def encode_with_truncation(examples):
    """Mapping function to tokenize the sentences passed with truncation"""
    return tokenizer(examples["text"], truncation=True, padding="max_length",
                                     max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
    """Mapping function to tokenize the sentences passed without truncation"""
    return tokenizer(examples["text"], return_special_tokens_mask=True)

In [12]:
# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

In [13]:
# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

In [14]:
# test_dataset['text']

In [15]:
# print(train_dataset[:2])
len(train_dataset)  # 14769

14769

In [16]:
if truncate_longer_samples:
    # remove other columns and set input_ids and attention_mask as PyTorch tensors
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
    # remove other columns, and remain them as Python lists
    test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
    train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [17]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

In [18]:
# max_length = 64

In [19]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

In [20]:
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
    train_dataset1 = train_dataset.map(group_texts, batched=True,
                                                                        desc=f"Grouping texts in chunks of {max_length}")
    test_dataset1 = test_dataset.map(group_texts, batched=True,
                                                                    desc=f"Grouping texts in chunks of {max_length}")
    # convert them from lists to torch tensors
    train_dataset1.set_format("torch")
    test_dataset1.set_format("torch")

In [21]:
# len(train_dataset), len(test_dataset)  # (14769, 1642)
len(train_dataset1), len(test_dataset1)  # (2171, 225)

(2423, 251)

In [22]:
len(train_dataset1['input_ids'][1])

512

In [23]:
for i in range(2):
    print(train_dataset1['input_ids'][i])
    print(tokenizer.decode(train_dataset1['input_ids'][i]))

tensor([  37, 1075, 1198,   39,  891, 7003,   16, 6244, 2185, 3629, 3826, 1623,
         447,  300,  428,  750,  398, 2129, 1185, 7661, 2781,   16, 2969,  547,
         414, 5272,  401, 2338, 7140, 2159,  645,  595,  616,  551,   16,  863,
        1527,  389,  960,  676,   18,   21,   18, 1399, 1200, 1558, 1300, 1875,
          16,   37, 4342,  283,   39,   12,   41,   13,   22,  611,   24, 1288,
           9,  518,  578, 1401,  784,   17, 5718,  678, 1149,  457,   16,   37,
        4342,  272,   39,   12,   42,   13,   24,  611, 4296,    9,  518,  578,
        1401,  784,   17, 4488,  457,   16,  412,   37, 4342,  278,   39,   12,
          43,   13,   28,  283,  611,   29,  265, 1288,    9,  518,  746,   12,
          49,   13,  498, 5165,  578, 1401,  758, 1059,   18,   22,   18, 1399,
        1200, 4682, 3097,   24,  611, 3359,  383,  539,  510,   16, 2314, 1826,
        3097,   25,  611, 4187,  539,  954, 1059,   18,   23,   18, 1399, 1200,
        1558, 1300, 1658,  382,  730,  9

### Loading the Model

In [24]:
vocab_size

8000

In [25]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Pre-Training - Additional

In [ ]:
# load the model checkpoint  : 8.30 실행 코드, max_len = 64로 학습된 모델을 로드하여 512로 변경 후 추가 학습
# model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-7500"))  # max_len = 64로 pre-train한 모델 로드 (8.22 완료)
# model.config.max_length = 512  # 512로 추가 학습하기 위해 모델 설정 변경

In [37]:
model = BertForMaskedLM.from_pretrained(os.path.join('c09k_pretrained_bert_512', "checkpoint-15000"))  # max_len = 512로 추가 pre-train한 모델 로드 (8.30 완료)

loading configuration file c09k_pretrained_bert_512/checkpoint-15000/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert_512/checkpoint-7000",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert_512/checkpoint-15000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c

In [38]:
# # initialize the model with the config
# model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
# model = BertForMaskedLM(config=model_config)

In [39]:
# model

In [40]:
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file c09k_pretrained_bert/config.json
Model config BertConfig {
  "_name_or_path": "c09k_pretrained_bert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 512,
  "max_position_embeddings": 512,
  "model_max_length": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sep_token": "[SEP]",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 30522
}



vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json


In [41]:
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [42]:
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

[{'score': 0.35562488436698914, 'token': 1193, 'token_str': '발광', 'sequence': '인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.2317202389240265, 'token': 1125, 'token_str': '일렉트로크로믹', 'sequence': '인광성 유기 금속 이리듐 착체, 일렉트로크로믹 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.11168079078197479, 'token': 737, 'token_str': '표시', 'sequence': '인광성 유기 금속 이리듐 착체, 표시 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.05203559994697571, 'token': 3316, 'token_str': '광기능', 'sequence': '인광성 유기 금속 이리듐 착체, 광기능 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.03952431306242943, 'token': 745, 'token_str': '광학', 'sequence': '인광성 유기 금속 이리듐 착체, 광학 소자, 발광 장치, 전자 기기, 및 조명 장치'}]
인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치, co

In [43]:
model_path

'c09k_pretrained_bert'

In [44]:
# Num examples = 19400
# Num Epochs = 50
# Total optimization steps = 3750 = 750*50 = 
# 로드한 모델 추가학습
training_args = TrainingArguments(
    output_dir='c09k_pretrained_bert_512_2',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
    overwrite_output_dir=False,     # 원래 True였으나 변경 (8.30) 
    num_train_epochs=3000.,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=8, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=4,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=8,  # evaluation batch size
    logging_steps=3000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=3000,
    resume_from_checkpoint=True,  # 체크포인트를 이어서 학습 가능할지 확인을 위해 새로 추가(8.30)
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
#     save_total_limit=5,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 3000
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset1,
    eval_dataset=test_dataset1,
)

In [46]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/hdh/PycharmProjects/bert_pretrain/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2423
  Num Epochs = 3000
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 225000


Step,Training Loss,Validation Loss
3000,1.025300,1.588253
6000,0.737500,1.519794
9000,0.537100,1.499150
12000,0.410600,1.501127
15000,0.329600,1.495798
18000,0.286800,1.503644
21000,0.241900,1.515726
24000,0.208700,1.555980
27000,0.182400,1.583822
30000,0.161200,1.549630


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-3000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-3000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-3000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-6000
Configuration saved in c09k_pretrained_b

  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-45000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-45000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-45000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-48000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-48000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-48000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not 

***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-90000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-90000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-90000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-93000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-93000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-93000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: 

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-135000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-135000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-135000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-138000
Configuration saved in c09k_pret

***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-177000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-177000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-177000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 251
  Batch size = 8
Saving model checkpoint to c09k_pretrained_bert_512_2/checkpoint-180000
Configuration saved in c09k_pretrained_bert_512_2/checkpoint-180000/config.json
Model weights saved in c09k_pretrained_bert_512_2/checkpoint-180000/pytorch_model.bin


KeyboardInterrupt: 

### Masked-LM 성능 확인
* 모델과 토크나이저를 한번은 로드 해줘야 하네. 그렇지 않으면 토크나이저가 CPU에 로드되어 있는 것으로 인식하여 에러 발생

In [56]:
# load the model checkpoint
model1 = BertForMaskedLM.from_pretrained(os.path.join('c09k_pretrained_bert_512', "checkpoint-7000"))
# load the tokenizer
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
tokenizer1 = BertTokenizerFast.from_pretrained(model_path, vocab_size=8000, local_files_only=True)
fill_mask = pipeline("fill-mask", model=model1, tokenizer=tokenizer1)
# perform predictions
# 인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, 전자 기기, 및 조명 장치
# 본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 유기 발광 소자에 관한 것이다
examples = [
    "인광성 유기 금속 이리듐 착체, [MASK] 소자, 발광 장치, 전자 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 [MASK] 및 이를 포함하는 유기 발광 소자에 관한 것이다",
    "인광성 유기 금속 이리듐 착체, 발광 소자, 발광 장치, [MASK] 기기, 및 조명 장치",
    "본 명세서는 화학식 1로 표시되는 화합물 및 이를 포함하는 [MASK] 발광 소자에 관한 것이다",
]
for example in examples:
    print(fill_mask(example))
    for prediction in fill_mask(example):
        print(f"{prediction['sequence']}, confidence: {prediction['score']}")
    print("="*50)

loading configuration file c09k_pretrained_bert_512/checkpoint-7000/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8000
}

loading weights file c09k_pretrained_bert_512/checkpoint-7000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at c09k_pretrained_bert_512/checkpoint-7000.
If your task is similar 

vocab_file vocab.txt
tokenizer_file tokenizer.json
added_tokens_file added_tokens.json
special_tokens_map_file special_tokens_map.json
tokenizer_config_file tokenizer_config.json
[{'score': 0.06595180928707123, 'token': 16, 'token_str': ',', 'sequence': '인광성 유기 금속 이리듐 착체,, 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.06593338400125504, 'token': 678, 'token_str': '유기', 'sequence': '인광성 유기 금속 이리듐 착체, 유기 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.04995021969079971, 'token': 805, 'token_str': '금속', 'sequence': '인광성 유기 금속 이리듐 착체, 금속 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.01198280043900013, 'token': 450, 'token_str': '전기', 'sequence': '인광성 유기 금속 이리듐 착체, 전기 소자, 발광 장치, 전자 기기, 및 조명 장치'}, {'score': 0.01068327110260725, 'token': 412, 'token_str': '및', 'sequence': '인광성 유기 금속 이리듐 착체, 및 소자, 발광 자

### 그 외 참고용 자료

In [ ]:
# 명선 책임의 문장 분리 코드(3글자 이후 마침표가 등장하는 경우에 간혹 분리되는 경우가 있다. 미사용)
# REG_SENT_KO=r'([ㄱ-ㅣ가-힣]+[.]|[\n]|[:;!?])'
# REG_SENT_EN=r'([a-zA-Z]+[.]\s|[\n]|[:;!?])'

# def split_sentence(doc, regex):
#     s = 0
#     for m in re.finditer(regex, doc):
#         sent = doc[s:m.end()].strip()
#         s = m.end()
#         if not sent:
#             continue
#         yield sent

#     if s < len(doc):
#         sent = doc[s:].strip()
#         if sent:
#             yield sent

### using personal dataset
* https://huggingface.co/docs/datasets/dataset_script

In [ ]:
# # if you have huge custom dataset separated into files
# # load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")

### 문장 클린징 처리 전/후 비교

In [8]:
print(dataset['text'][0:5])
print(dataset['text'][-5:])
# 문장 클린징 처리 전
# ['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 &lt; x &lt; 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1@@발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2@@발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
# ['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며,', '산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.', '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.', '１０〜２． 54의 범위내인 원소 B만이 함유 되어', '바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']
# 처리 후에는 1문장으로 처리될 데이터가 무려 5문장으로 나눠진다!

# 문장 클린징 처리 후
# ['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 x 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1 발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2 발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
# ['전기 화학 디바이스 및 그것을 형성하는 방법', '전기 화학 디바이스에 대해서, 개시한다.전기 화학 디바이스는, 제 1의 투명 도전층, 제 1의 투명 도전층 위에 있는 에렉트로 크로믹층, 에렉트로 크로믹층 위에 있는 대향 전극층, 제 2의 투명 도전층을 갖추어 23°C으로 0.68초/mm이하의 스위칭 속도 파라미터를 가진다.', '제 1의 투명 도전층과 전기 제 1의 투명 도전층 위에 있는 음극 전기 화학층과 전기 음극 전기 화학층 위에 있는 양극 전기 화학층과 제 2의 투명 도전층과 (을)를 갖추는 전기 화학 디바이스이며, 전기 전기 화학 디바이스는, 23°C으로 0.68초/mm이하인 제 1의 스위칭 속도 파라미터, -20°C으로 1.0초/mm이하인 제 2의 스위칭 속도 파라미터, 1.5/log(오옴) 미만인 log|Z|의 impedance파라미터, 또는 -20°C으로 8%미만인 착색 투과률 파라미터 중 적어도 1개를 가지는, 전기 화학 디바이스.', '이산화 바나듐 함유 입자, 사모 크로믹 필름 및 이산화 바나듐 함유 입자의 제조 방법', 
#  '사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.65〜2.05의 범위내인 원소 A와 폴링의 전기 음성도가 2.10〜2.54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']

['이산화티탄-염화리튬 전기변색물질, 이를 이용한전기변색장치용 파우더 및 필름전극의 제조방법', '본 발명은 이산화티탄-염화리튬 전기변색물질 및 이를 이용한 전기변색장치용 파우더 및 필름전극의 제조방법에 관한 것으로, 이산화티탄에 염화리튬이 균일하게 혼합된 물질을 합성하여 제조되며 상기 이산화티탄-염화리튬에서 티탄 대 리튬의 몰 함량 비율이 1 : x (0.5 x 2) 범위를 갖는 이산화티탄-염화리튬을 졸-겔 방법으로 합성함으로써 표면적이 극대화되는 다공성화 또는 그 물질 내에 존재하는 높은 함량의 리튬 이온으로 인하여 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 매우 뛰어난 발명인 것이다.', '이산화티탄-염화리튬 파우더에 아세틸아세톤이 포함된 증류수를 첨가하여 끈적거리는 상태가 될 때까지 교반하는 단계; 상기 용액에 증류수와 트리톤 X-100을 첨가하여 완전히 균일한 용액이 될 때까지 다시 교반하는 단계; 및 상기 용액을 전도성 유리인 ITO 위에 얇게 입힌 후, 상온에서 건조한 다음 열처리하는 단계를 포함함을 특징으로 하는 전기변색장치용 이산화티탄-염화리튬 필름전극의 제조방법.', '1 발명은 상기의 제문제점을 감안하여 안출한 것으로, 본 발명에서는 이산화티탄-염화리튬을 새롭게 합성하여 전기변색 물질로서 실용화함에 그 목적이 있다.', '2 발명의 다른 목적은 전기변색 감응시간을 획기적으로 줄이고 전기변색 효율을 크게 향상시킬 수 있는 이산화티탄-염화리튬 전기변색물질을 제공하는데 있다.']
['전기 화학 디바이스 및 그것을 형성하는 방법', '전기 화학 디바이스에 대해서, 개시한다.전기 화학 디바이스는, 제 1의 투명 도전층, 제 1의 투명 도전층 위에 있는 에렉트로 크로믹층, 에렉트로 크로믹층 위에 있는 대향 전극층, 제 2의 투명 도전층을 갖추어 23°C으로 0.68초/mm이하의 스위칭 속도 파라미터를 가진다.', '제 1의 투명 도전층과 전기 제 1의 투명 도전층 위에 있는 음극 전기 화학층과 전기 음극 전기 화학층 

In [9]:
# """
# ['사모 크로믹성을 가지는 이산화 바나듐 함유 입자이며, 산소 및 바나듐 이외의 원소로서, 폴링의 전기 음성도가 1.',
#  '６５〜２． 05의 범위내인 원소 A와 폴링의 전기 음성도가 2.',
#  '１０〜２． 54의 범위내인 원소 B만이 함유 되어 바나듐(100 atom%)에 대해, 전기 원소 A의 총함유량이 0.5~20 atom%의 범위내이며, 한편, 전기 원소 B의 총함유량이 0.05~20 atom%의 범위내인 이산화 바나듐 함유 입자.']
#  위와 같이 소수점 앞 뒤로 공백이 존재하는데, 이로 인해 소수점에서 문장 분리가 되버린다. 오류 조치가 필요
#  """
# 아래와 같이 구두점 앞뒤 블랭크를 제거하여 일단 구분
# row[col] = unicodedata.normalize('NFKC', row[col])
# https://blog.naver.com/PostView.nhn?blogId=duswl0319&logNo=221516880642&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
# row[col] = row[col].replace(' . ','.')
# row[col] = row[col].replace(' ． ','.')
# row[col] = row[col].replace('． ','.')
# row[col] = row[col].replace('. ','.')
# row[col] = row[col].replace('  ',' ')

In [ ]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
    """Utility function to save dataset text to disk,
    useful for using the texts to train the tokenizer 
    (as the tokenizer accepts files)"""
    with open(output_filename, "w") as f:
        for t in dataset["text"]:
            print(t, file=f)

In [ ]:
# # train_dataset.features.keys()
# # dict_keys(['text', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'])
# total_length = len(list(chain(train_dataset['text'])))  # total_length = 14769
# max_length  # 512
# total_length = (total_length // max_length) * max_length
# # total_length  # 14336, total_length // max_length = 28.8457..., 28 * 512 = 14336